In [130]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import entropy
from sklearn.datasets import load_breast_cancer


In [131]:
# import some data to play with
iris = datasets.load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.33, random_state=42)


# normalize the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)
# def change_weights(X_train,y_train,X_test,y_test,weights):

In [132]:
pk = np.array([1/5, 2/5, 2/5])  # fair coin
H = entropy(pk)
print(H)

1.0549201679861442


In [133]:

def calculate_probabilities(list_labels, uniq_labels):
    '''
    Author: Sara Nassar 
    this function calculates the probabilities of each label in the list of labels
    it is calculated by number of labels in class A/all labels
    number of labels in class B/all labels
    and so on
    '''
    
    # A dictionary to store the probabilities
    probabilities = dict.fromkeys(uniq_labels, 0)
    
    # Total number of labels
    total_labels = len(list_labels)
    
    for label in uniq_labels:
        # Counting the number of times the label occurs in the list
        count = list_labels.count(label)
        
        # Calculating the probability of the label
        probability = count / total_labels
        
        # Storing the calculated probability in the dictionary
        probabilities[label] = probability
        
    return probabilities    
    
    
# test your function
list_labels=[1,2,0,1,2,0]
uniq_labels=[0,1,2]
print(calculate_probabilities(list_labels,uniq_labels))
# this should print somehting like 0.33,0.33,0.33


{0: 0.3333333333333333, 1: 0.3333333333333333, 2: 0.3333333333333333}


In [134]:

def calc_entropy_from_probabilities(list_probas):
    '''
    Author: Sara Nassar 
    list_probas is the list of probabiities
    the formula for entropy is
    sum(-proba*log(proba))
    
    '''
    
    entropy_value = 0

    for proba in list_probas:
        # If the probability is not zero
        if proba != 0:
            entropy_value += -proba * np.log(proba)
     
    return entropy_value


# test your function
list_probas=[1/5, 2/5, 2/5]
print(calc_entropy_from_probabilities(list_probas))
# above should print 1.054...

1.0549201679861442


In [135]:
def information_gain(old_entropy, new_entropies, count_items):
    '''
    Author: Sara Nassar 
    from the list of new entropies, calculate the overall new entropy
    
    formula is something like:
    overall_new_entropy = entropy1*proportion1 + entropy2*proportion2+ entropy3*proportion3 ...
    
    igain=old_entropy-overall_new_entropy
    '''
    
    overall_new_entropy = 0
    
    # Calculating the total number of items
    total_items = sum(count_items.values())
    
    for i in range(len(new_entropies)):
        if i in count_items:
            # Calculating the proportion of items in the current partition
            proportion = count_items[i] / total_items
        
            # Adding the entropy of the current partition weighted by its proportion to the overall new entropy
            overall_new_entropy += new_entropies[i] * proportion
        
    # Calculating the information gain
    information_gain = old_entropy - overall_new_entropy
    
    return information_gain


old_entropy=1
new_entropies=[0,0.65]
count_items=[4,6]
count_items = {i: count_items[i] for i in range(len(count_items))}
print(information_gain(old_entropy,new_entropies,count_items)) # should print 0.61

  

0.61


In [136]:

def initialize_weights(number_features):
    '''
    the first set of weights corresponding to the features
    For now, it defaults to 2
    '''
    
    weights=np.array([np.random.uniform() for i in range(number_features)])
    return weights
    

In [137]:
num_feats=X_train.shape[1]
print(initialize_weights(num_feats))

[0.29332121 0.11841787 0.90289701 0.20699767]


In [138]:
def get_entropy_from_groups(new_entropies,count_items):
    overall_new_entropy = 0
    
    # Calculating the total number of items
    total_items = sum(count_items)
    
    for i in range(len(new_entropies)):
        # Calculating the proportion of items in the current partition
        proportion = count_items[i] / total_items
        
        # Adding the entropy of the current partition weighted by its proportion to the overall new entropy
        overall_new_entropy += new_entropies[i] * proportion
        
    return overall_new_entropy    

def get_entropy(threshold,res,y_test):

    # make two groups
    group1=[]
    group2=[]

    for i in range(res.shape[0]):
        if res[i]<threshold:
            group1.append(y_test[i])
        else:
            group2.append(y_test[i])


    proba_gr1=calculate_probabilities(group1,np.unique(group1).tolist())
    proba_gr1=list(proba_gr1.values()) 
    entropy_group1=calc_entropy_from_probabilities(proba_gr1)
    count_group1=len(proba_gr1)

    proba_gr2=calculate_probabilities(group2,np.unique(group2).tolist())
    proba_gr2=list(proba_gr2.values()) 
    entropy_group2=calc_entropy_from_probabilities(proba_gr2)
    count_group2=len(proba_gr2)

    new_entropies=[entropy_group1,entropy_group2]
    count_items=[count_group1,count_group2]
    overall_new_entropy=get_entropy_from_groups(new_entropies,count_items)
    return overall_new_entropy


### Task4: PSO Implementation

#### Modified the entropy function to get a vector of entropies for n particles

In [139]:
 def objective_fn(param1, param2, X, y):
    '''
    Author: Sara Nassar
    param1 and param2 are the parameters that we want to optimize
    say param1 is the weight vector and  param2 is the threshold
    '''
    # Multiply the weights with each feature and calculate the sum
    res = np.sum(X * param1, axis=1)
    
    # Calculate entropy using the get_entropy function
    entropy = get_entropy(param2, res, y)
    
    return entropy   


In [140]:
def objective_fn_vector(params1, params2, X, y):
    '''
    Author: Sara Nassar
    params1 is an array of weight vectors
    params2 is an array of thresholds
    '''
    results = []
    for i in range(params1.shape[0]):
        param1 = params1[i] # get ith set of weights
        param2 = params2[i] # get ith threshold
        # call the objective_fn above to get the entropy
        res = objective_fn(param1, param2, X, y)
        #print(param2,res)
        results.append(res)
    
    return np.array(results)

In [141]:
### Below we just randomly assign 100 particles and see if we can find the global minimum.
### THis is just to check

In [142]:
'''
Author: Sara Nassar
Below we just randomly assign 100 particles and see if we can find the global minimum.
'''
num_particles = 100
num_features = X_train.shape[1]

params1 = []
for i in range(num_particles):
    weights = initialize_weights(num_features)
    params1.append(weights)

    
# we have a list of 100 weight vectors (params1) and 100 thresholds (params2)
# convert them to array
params1 = np.array(params1)
params2 = np.random.uniform(size=num_particles)

print("Shape of params 1 (weights):", params1.shape)
print("Shape of params 2 (thresholds):", params2.shape)


Shape of params 1 (weights): (100, 4)
Shape of params 2 (thresholds): (100,)


In [143]:
z = objective_fn_vector(params1, params2, X_train, y_train)
# Find the global minimum that is using the minimum if params1 and params2
global_min_idx = np.argmin(z)
param1_min = params1[global_min_idx] # use z.argmin() to access best params1 (global_min_weight)
param2_min = params2[global_min_idx] # use z.argmin() to access best params2 (global_min_threshold)

print("param1_min",param1_min,"param2_min",param2_min)

param1_min [0.8252117  0.04036757 0.78453879 0.84722558] param2_min 0.7756162615317937


In [144]:
# Hyper-parameter of the algorithm
c1 = c2 = 0.1
w1 = np.array([np.random.uniform() for i in range(X_train.shape[1])])
w2 = 0.8 

# Create particles
n_particles = 20
np.random.seed(100)
params1 = np.array([initialize_weights(X_train.shape[1]) for i in range(n_particles)])
params2 = np.random.uniform(size=n_particles)

params1 = np.array(params1)
params2 = np.array(params2)

print("params1 shape is ",params1.shape,"params2 shape is ",params2.shape)

params1 shape is  (20, 4) params2 shape is  (20,)


In [145]:
# Define velocity of each weight of every particle
V_param1 = np.array([initialize_weights(X_train.shape[1]) for i in range(n_particles)])

# Define velocity of each threshold of every particle
V_param2 = np.random.uniform(size=n_particles)

# Initialize objective values
pbest = (np.array([param.copy() for param in params1]), params2.copy())
pbest_obj = objective_fn_vector(params1, params2, X_train, y_train)
gbest = (params1[pbest_obj.argmin()], params2[pbest_obj.argmin()])
gbest_obj = pbest_obj.min()

print("pbest obj value for 20 particles are as follows",pbest_obj)
print("gbest obj value among all 20 particles is as follows",gbest_obj)
# Note that gbest_obj should be the minimum of all pbest_obj

pbest obj value for 20 particles are as follows [0.67013703 0.82232957 0.64329013 0.70573338 0.73886477 0.82232957
 1.09729975 0.5237323  0.77244152 0.4620281  0.86703698 0.81919055
 1.09729975 0.81919055 0.73355763 0.74030523 0.82232957 0.73805779
 1.09729975 0.68309963]
gbest obj value among all 20 particles is as follows 0.4620281046196322


### the update function

In [146]:
def update():
    "Function to do one iteration of particle swarm optimization"
    global V_param1, V_param2, params1, params2, pbest, pbest_obj, gbest, gbest_obj
    # these have been already initialized in the previous cells
    
    # Update params
    r11, r12, r2 = np.random.rand(3)
    V_param1 = w1 * V_param1 + c1 * r11 * (pbest[0] - params1) + c2 * r2 * (gbest[0] - params1)
    V_param2 = w2 * V_param2 + c1 * r12 * (pbest[1] - params2) + c2 * r2 * (gbest[1] - params2)    
   #V = w * V + c1*r11*(pbest - params1) + c2*r2*(gbest.reshape(-1,1)-X)
    params1 = params1 + V_param1
    params2 = params2 + V_param2
    
    obj = objective_fn_vector(params1, params2, X_train, y_train)
    for i in range(pbest[0].shape[0]):
        if pbest_obj[i] >= obj[i]:
            pbest[0][i] = params1[i]
            pbest[1][i] = params2[i]
            pbest_obj[i] = obj[i]
            
    gbest = (params1[pbest_obj.argmin()], params2[pbest_obj.argmin()])
    gbest_obj = pbest_obj.min()

In [147]:
for i in range(100):
    update()

num_features = X_train.shape[1]
param1_min = initialize_weights(num_features) 

print("PSO found best solution at f({})={}".format(gbest, gbest_obj))
print("Global optimal at f({})={}".format([param1_min,param2_min], objective_fn(param1_min, param2_min, X_train, y_train)))


PSO found best solution at f((array([1.01229026, 0.59462971, 1.88398165, 0.60763938]), 1.248160629484322))=0.4161039895073432
Global optimal at f([array([0.55252942, 0.1611271 , 0.56854896, 0.48570987]), 0.7756162615317937])=0.618501821515296


In [148]:
# import some data to play with
#load the breast cancer dataset 
bcan = load_breast_cancer()
X = bcan.data
y = bcan.target

X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.33, random_state=42)


# normalize the data
scaler = MinMaxScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)
print("Shape of training data is ",X_train.shape)

Shape of training data is  (381, 30)


In [149]:
n_particles = 100
params1 = []
params2 = []
for i in range(n_particles):
    params1.append(initialize_weights(X_train.shape[1]))
    params2.append(np.random.uniform())
params1 = np.array(params1)
params2 = np.array(params2)
print("Shape of params 1 (weights)",params1.shape)
print("Shape of params 2 (thresholds)",params2.shape)

Shape of params 1 (weights) (100, 30)
Shape of params 2 (thresholds) (100,)


In [150]:
z = objective_fn_vector(params1, params2, X_train, y_train)
global_min_idx = z.argmin()
param1_min = params1[global_min_idx]
param2_min = params2[global_min_idx]
global_min_obj_val = z[global_min_idx]
print("param1_min",param1_min,"param2_min",param2_min)

param1_min [0.12751954 0.54369217 0.20049061 0.67016086 0.55811222 0.23237832
 0.51558849 0.31499197 0.84659255 0.44737628 0.10008522 0.90159209
 0.85608825 0.28632987 0.25901477 0.06637137 0.31763167 0.051843
 0.94418832 0.7172173  0.5536592  0.35974477 0.1591823  0.43295804
 0.27936218 0.96103763 0.09813216 0.40699555 0.00837645 0.56805893] param2_min 0.5766097683672401


In [151]:
# Hyper-parameter of the algorithm
c1 = c2 = 0.1
w1 = np.array([np.random.uniform() for i in range(X_train.shape[1])])
w2 = 0.8 
# Create particles
n_particles = 20
params1 = []
params2 = []
for i in range(n_particles):
    params1.append(initialize_weights(X_train.shape[1]))
    params2.append(np.random.uniform())

params1=np.array(params1)
params2=np.array(params2)

print("params1 shape is ",params1.shape,"params2 shape is ",params2.shape)


params1 shape is  (20, 30) params2 shape is  (20,)


In [152]:
# define velocity of each weight of every particle
n_particles = 20
V_param1 = []
V_param2 = []
for i in range(n_particles):
    V_param1.append(initialize_weights(X_train.shape[1]))
    V_param2.append(np.random.uniform())

V_param1=np.array(V_param1)
V_param2=np.array(V_param2)

In [153]:
# Initialize objective values
pbest = (params1.copy(), params2.copy())
pbest_obj = objective_fn_vector(params1, params2, X_train, y_train)

gbest=(params1[pbest_obj.argmin()],params2[pbest_obj.argmin()])
gbest_obj = pbest_obj.min()

print("pbest obj value for 20 particles are as follows",pbest_obj)
print("gbest obj value among all 20 particles is as follows",gbest_obj)
# note that gbest_obj should be the minimim of all pbest_obj

pbest obj value for 20 particles are as follows [0.6643461 0.6643461 0.6643461 0.6643461 0.6643461 0.6643461 0.6643461
 0.6643461 0.6643461 0.6643461 0.6643461 0.6643461 0.6643461 0.6643461
 0.6643461 0.6643461 0.6643461 0.6643461 0.6643461 0.6643461]
gbest obj value among all 20 particles is as follows 0.6643460978641622


In [154]:
for i in range(100):
    update()
print("PSO found best solution at f({})={}".format(gbest, gbest_obj))
print("Global optimal at f({})={}".format([param1_min,param2_min], objective_fn(param1_min, param2_min, X_train, y_train)))

PSO found best solution at f((array([0.81478416, 2.16135537, 0.80949853, 3.47002409, 1.53346952,
       3.21786451, 4.69918283, 9.56614919, 1.34898383, 1.05069426,
       0.27373452, 0.35291515, 1.29812582, 2.00260603, 0.70941125,
       4.12196654, 2.14378816, 1.07077156, 2.79188037, 1.16440568,
       3.67590249, 0.20751861, 0.72134805, 0.32680045, 0.46749854,
       0.53694585, 0.43963777, 0.399247  , 0.34559994, 1.23349266]), 1.8817097915254228))=0.6643460978641622
Global optimal at f([array([0.12751954, 0.54369217, 0.20049061, 0.67016086, 0.55811222,
       0.23237832, 0.51558849, 0.31499197, 0.84659255, 0.44737628,
       0.10008522, 0.90159209, 0.85608825, 0.28632987, 0.25901477,
       0.06637137, 0.31763167, 0.051843  , 0.94418832, 0.7172173 ,
       0.5536592 , 0.35974477, 0.1591823 , 0.43295804, 0.27936218,
       0.96103763, 0.09813216, 0.40699555, 0.00837645, 0.56805893]), 0.5766097683672401])=0.6643460978641622


In [155]:
max_tree_size=128
all_optimized_weights_list=[None for i in range(max_tree_size)]
all_optimized_thresh_list=[None for i in range(max_tree_size)]
all_dataset_sizes_list=[None for i in range(max_tree_size)]
all_IG_list=[None for i in range(max_tree_size)]
        

In [156]:
X_train.shape

(381, 30)

In [157]:
def update(V_param1,V_param2, params1,params2, pbest, pbest_obj, gbest, gbest_obj):
    "Function to do one iteration of particle swarm optimization"
    # these have been already initialized in the previous cells
    
    # Update params
    r11,r12, r2 = np.random.rand(3)
    V_param1=w1*V_param1+c1*r11*(pbest[0] - params1)+ c2*r2*(gbest[0]-params1)
    V_param2=w2*V_param2+c1*r12*(pbest[1] - params2)+ c2*r2*(gbest[1]-params2)    
#     V = w * V + c1*r11*(pbest - params1) + c2*r2*(gbest.reshape(-1,1)-X)
    params1 = params1 + V_param1
    params2 = params2 + V_param2
    
    obj = objective_fn_vector(params1, params2, X_train, y_train)
    for i in range(pbest[0].shape[0]):
        if pbest_obj[i]>=obj[i]:
            
            pbest[0][i]=params1[i] # update pbest[0][i] with value of params1[i]
            pbest[1][i]=params2[i] # update pbest[1][i] 
            pbest_obj[i]=obj[i]    # also update pbest_obj[i]

            
    gbest=(params1[pbest_obj.argmin()],params2[pbest_obj.argmin()]) # update gbest to contain the best from params1 and params 2
    gbest_obj = pbest_obj.min() # update gbest to get the minimum of pbest_obj

In [158]:
max_tree_size=128
all_optimized_weights_list=[None for i in range(max_tree_size)]
all_optimized_thresh_list=[None for i in range(max_tree_size)]
all_dataset_sizes_list=[None for i in range(max_tree_size)]
all_IG_list=[None for i in range(max_tree_size)]

threshold_dict = {}

def find_best_params(train_x,train_y,test_x,test_y,node_number):
      
    '''
    recursive function to get the best set of weights
    '''
    print("node_number",node_number,"data shape",train_x.shape)
    # exit condition 1: if the node_number is more than the maximum tree size, return
    if node_number>=max_tree_size:
        return all_optimized_weights_list, all_optimized_thresh_list, all_dataset_sizes_list, all_IG_list
    # exit condition 2: if the training dataset has one or less rows, return
    if train_x.shape[0]<=1:
        return all_optimized_weights_list, all_optimized_thresh_list, all_dataset_sizes_list, all_IG_list
    # exit condition 3: if the train_y has values from only one class (only 0s or only 1s and so on)
    if len(np.unique(train_y))==1:
        return all_optimized_weights_list, all_optimized_thresh_list, all_dataset_sizes_list, all_IG_list


    # Hyper-parameter of the algorithm
    c1 = c2 = 0.1
    w1 = np.array([np.random.uniform() for i in range(X_train.shape[1])])
    w2 = 0.8 
    # Create particles
    n_particles = 20
    np.random.seed(100)
    params1=[initialize_weights(X_train.shape[1]) for i in range(n_particles)] # a vector of shape n_particles,n_features
    # call the initialize_weights function above

    params2=[np.random.uniform() for i in range(n_particles)]# a vector of shape n_particles
    # use the np.random.uniform() function

    params1=np.array(params1)
    params2=np.array(params2)

    # define velocity of each weight of every particle
    V_param1 = [initialize_weights(X_train.shape[1])*0.1 for i in range(n_particles)] # shape is same as params1
    # once again can use initialize_weights function

    #define velocity of each threshold of every particle
    V_param2 = np.array([np.random.uniform()*0.1 for i in range(n_particles)])# shape is same as params2
    # once again use np.random.uniform

        # Initialize objective values
    pbest = (params1,params2)
    pbest_obj = objective_fn_vector(params1, params2, X_train, y_train)
    gbest=(params1[pbest_obj.argmin()],params2[pbest_obj.argmin()])
    gbest_obj = pbest_obj.min()

    new_ys = np.dot(train_x, gbest[0])
    # normalize the new_ys
    new_ys = (new_ys - np.min(new_ys)) / (np.max(new_ys) - np.min(new_ys))

    for i in range(100):
        update(V_param1, V_param2, params1, params2, pbest, pbest_obj, gbest, gbest_obj)
        # calculate new_ys inside the loop
        new_ys = np.dot(train_x, gbest[0])
        # normalize the new_ys
        new_ys = (new_ys - np.min(new_ys)) / (np.max(new_ys) - np.min(new_ys))
        # calculate the threshold using the 50th percentile of new_ys
        threshold = np.percentile(new_ys, 50) 
        #Half of the new_ys values will be greater than the threshold and half will be lower 

    unique, counts = np.unique(train_y, return_counts=True)
    count_items = dict(zip(unique, counts))

    # add the achieved optimized values to the lists    
    all_optimized_weights_list[node_number]=gbest[0]
    all_optimized_thresh_list[node_number]=threshold
    all_dataset_sizes_list[node_number]=train_x.shape[0]
    all_IG_list[node_number] = information_gain(train_y, new_ys, count_items)

    # Save threshold value to dictionary with node number as key
    threshold_dict[node_number] = threshold

    # chop the data into two parts: left
    train_x_left = train_x[new_ys >= threshold]
    train_y_left = train_y[new_ys >= threshold]
    left_child_node_num = node_number * 2 + 1

    # chop the data into two parts: right
    train_x_right = train_x[new_ys < threshold]
    train_y_right = train_y[new_ys < threshold]
    right_child_node_num = node_number * 2 + 2

    # exit condition 4: return if information gain is 0
    if np.allclose(information_gain(train_y, new_ys, count_items), 0):
        return

    print("Left",train_x_left.shape)
    print("Right",train_x_right.shape)
    # make the recursion call for left
    find_best_params(train_x_left, train_y_left, test_x, test_y, left_child_node_num)

    # make the recursion call for right
    find_best_params(train_x_right, train_y_right, test_x, test_y, right_child_node_num)

   # return all_optimized_weights_list, all_optimized_thresh_list, all_dataset_sizes_list, all_IG_list, threshold_dict


In [159]:
node_number=0
find_best_params(X_train,y_train,X_test,y_test,node_number)

node_number 0 data shape (381, 30)
Left (191, 30)
Right (190, 30)
node_number 1 data shape (191, 30)
Left (96, 30)
Right (95, 30)
node_number 3 data shape (96, 30)
Left (48, 30)
Right (48, 30)
node_number 7 data shape (48, 30)
Left (24, 30)
Right (24, 30)
node_number 15 data shape (24, 30)
Left (12, 30)
Right (12, 30)
node_number 31 data shape (12, 30)
node_number 32 data shape (12, 30)
Left (6, 30)
Right (6, 30)
node_number 65 data shape (6, 30)
Left (3, 30)
Right (3, 30)
node_number 131 data shape (3, 30)
node_number 132 data shape (3, 30)
node_number 66 data shape (6, 30)
node_number 16 data shape (24, 30)
node_number 8 data shape (48, 30)
Left (24, 30)
Right (24, 30)
node_number 17 data shape (24, 30)
Left (12, 30)
Right (12, 30)
node_number 35 data shape (12, 30)
node_number 36 data shape (12, 30)
Left (6, 30)
Right (6, 30)
node_number 73 data shape (6, 30)
node_number 74 data shape (6, 30)
Left (3, 30)
Right (3, 30)
node_number 149 data shape (3, 30)
node_number 150 data shape (3

In [160]:
all_optimized_thresh_list

[0.22441467237833546,
 0.2504066424997364,
 0.6070197815984363,
 0.16999422935636388,
 0.3870857323197978,
 0.40154230043943423,
 0.6953655013125566,
 0.18290547726336912,
 0.5243978726934442,
 0.4943942202059026,
 0.4199079226819821,
 0.5180354180900639,
 0.5023437334491847,
 0.490850755509969,
 None,
 0.26670256775705437,
 None,
 0.5365141637823322,
 None,
 0.5925103386771322,
 0.5738675330691457,
 0.4530154131226026,
 0.47245793766869004,
 0.39313282108823944,
 None,
 0.5966365029521083,
 0.4190398195863058,
 0.5754684486333098,
 None,
 None,
 None,
 None,
 0.5332669248888788,
 None,
 None,
 None,
 0.3492993970535968,
 None,
 None,
 0.328778668301407,
 0.49219480860396614,
 0.517777224609163,
 0.6177374815344943,
 0.5371743156798247,
 0.5430055792077073,
 0.688549740541477,
 0.3282434638570728,
 None,
 0.4347875268433696,
 None,
 None,
 0.7266951260247945,
 None,
 0.5651702088281381,
 None,
 0.5007759882503241,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 0.462721

In [161]:
#thresh=all_optimized_thresh_list[0]
#print(thresh)